In [14]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [98]:
import cv2
from IPython.display import HTML
import matplotlib.pyplot as plt
import matplotlib
import numpy as np
from pathlib import Path
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
from torchvision import transforms, datasets
from torchvision.datasets import ImageFolder, DatasetFolder
import torchvision.utils as vutils

from htools import hdir
from config import *
from models import BaseModel, conv_block, Discriminator
from utils import render_samples, show_img

In [17]:
# TEMPORARY, IMPORTED FROM CONFIG - JUST FOR EASY REFERENCE

bs = 64                # Batch size (paper uses 128).
img_size = 64          # Size of input (here it's 64 x 64).
workers = 2            # Number of workers for data loader.
input_c = 100          # Depth of input noise (1 x 1 x noise_dim). AKA nz.
ngf = 64               # Filters in first G layer.
ndf = 64               # Filters in first D layer.
lr = 2e-4              # Recommended learning rate of .0002.
beta1 = .5             # Recommended parameter for Adam.
nc = 3                 # Number of channels of input image.
ngpu = 1               # Number of GPUs to use.
sample_dir = 'samples' # Directory to store sample images from G. 
weight_dir = 'weights' # Directory to store model weights.
device = torch.device('cuda:0' if torch.cuda.is_available() and ngpu > 0 
                      else 'cpu')

In [213]:
class ResBlock(nn.Module):
    """Residual block to be used in CycleGenerator. Note that the relu or 
    leaky must still be applied on the output.
    """
    
    def __init__(self, c_in, num_layers=2, leak=.02):
        """
        Parameters
        -----------
        c_in: int
            # of input channels.
        num_layers: int
            Number of conv blocks inside the skip connection (default 2). 
            ResNet paper notes that skipping a single layer did not show
            noticeable improvements.
        """
        super().__init__()
        self.leak = leak
        self.layers = nn.ModuleList([conv_block(False, c_in, c_in, 3, 1, 1) 
                                     for i in range(num_layers)])
    
    def forward(self, x):
        x_out = x
        for layer in self.layers:
            x_out = F.leaky_relu(layer(x_out), self.leak)
        return x + x_out

In [215]:
class CycleGenerator(BaseModel):
    """CycleGAN Generator."""

    def __init__(self, img_c=3, ngf=64, leak=.02):
        """
        Parameters
        -----------
        img_c: int
            # of channels of input image.
        ngf: int
            # of channels in first convolutional layer.
        leak: float
            Slope of leaky relu where x < 0. Leak of 0 is regular relu.
        """
        super().__init__()
        self.leak = leak
        self.activation = nn.LeakyReLU(self.leak)

        # ENCODER
        # 3 x 64 x 64 -> 64 x 32 x 32
        deconv1 = conv_block(False, img_c, ngf, f=4, stride=2, pad=1)
        # 64 x 32 x 32 -> 128 x 16 x 16
        deconv2 = conv_block(False, ngf, ngf*2, 4, 2, 1)
        self.encoder = nn.Sequential(deconv1, 
                                     self.activation,
                                     deconv2,
                                     self.activation)

        # TRANSFORMER
        # 128 x 16 x 16 -> 128 x 16 x 16
        res1 = ResBlock(ngf*2, num_layers=2, leak=self.leak)
        # 128 x 16 x 16 -> 128 x 16 x 16
        res2 = ResBlock(ngf*2, 2, self.leak)
        self.transformer = nn.Sequential(res1,
                                         self.activation,
                                         res2,
                                         self.activation)

        # DECODER
        # 128 x 16 x 16 -> 64 x 32 x 32
        deconv1 = conv_block(True, ngf*2, ngf, f=4, stride=2, pad=1)
        # 64 x 32 x 32 -> 3 x 64 x 64
        deconv2 = conv_block(True, ngf, img_c, 4, 2, 1)
        self.decoder = nn.Sequential(deconv1, 
                                     self.activation,
                                     deconv2,
                                     nn.Tanh())

        # Module list of Sequential objects is helpful if we want to use 
        # different learning rates per group.
        self.groups = nn.ModuleList([self.encoder,
                                     self.transformer,
                                     self.decoder])

    def forward(self, x):
        for group in self.groups:
            x = group(x)
        return x

In [216]:
# class CycleDiscriminator(BaseModel):
    
#     def __init__(self, img_c=3, ndf=64):
#         super().__init__()
#         self.conv1 = conv_block(False, img_c, ndf, f=4, stride=2, pad=1)
        
#     def forward(self):
#         pass

In [217]:
G = CycleGenerator(img_c, ngf)

In [218]:
D = Discriminator(ndf)

In [219]:
len(G.dims())

24

In [225]:
G(x)

tensor([[[[ 0.5343, -0.1951,  0.9742, -0.7906],
          [ 0.3825, -0.0667,  0.6833, -0.9533],
          [-0.5036, -0.9747,  0.4012,  0.8723],
          [ 0.5653, -0.7044,  0.3609, -0.8299]],

         [[ 0.0013, -0.9774, -0.5825,  0.4348],
          [-0.4315,  0.0135,  0.9550, -0.1441],
          [-0.4449, -0.7311, -0.6828, -0.1035],
          [ 0.2147,  0.6039,  0.8773, -0.7132]],

         [[ 0.6585, -0.5894,  0.6888,  0.2868],
          [ 0.2329,  0.6130, -0.5606, -0.5357],
          [-0.0258, -0.2218,  0.2809, -0.6998],
          [-0.4595,  0.1366,  0.2749, -0.7526]]],


        [[[ 0.4473,  0.0556,  0.9664,  0.3324],
          [ 0.3801, -0.2523, -0.7848,  0.4776],
          [-0.6737,  0.9194, -0.7567,  0.5156],
          [-0.4637, -0.6024,  0.4821, -0.7558]],

         [[-0.8447, -0.8026,  0.5185,  0.1444],
          [-0.5484,  0.8674,  0.9145,  0.5140],
          [-0.4182, -0.8895,  0.9462, -0.6652],
          [ 0.6073,  0.8936,  0.0090, -0.3644]],

         [[ 0.8318, -0.8134,